# WetGrass analyzed using Edward (batch_size > 1)

In [1]:
import numpy as np
import tensorflow as tf

import edward as ed
import edward.models as edm
import edward.inferences as edi

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

In [2]:
import os
import sys
cur_dir_path = os.getcwd()
print(cur_dir_path)
os.chdir('../../')
qfog_path = os.getcwd()
print(qfog_path)
sys.path.insert(0,qfog_path)

/home/bob/Notebooks/Quantum/quantum-fog/jupyter-notebooks/inference_via_ext_software
/home/bob/Notebooks/Quantum/quantum-fog


In [3]:
# build BayesNet object bnet from bif file
import importlib
mm = importlib.import_module("jupyter-notebooks.inference_via_ext_software.ModelMaker")
from graphs.BayesNet import *
in_path = "examples_cbnets/WetGrass.bif"
bnet = BayesNet.read_bif(in_path, False)

In [4]:
# build model (with placeholders for Cloudy and WetGrass) from bnet
file_prefix = "examples_cbnets/WetGrass"
vtx_to_data = {'Cloudy': [1], "WetGrass": [0, 1]}
batch_size = 100
mm.ModelMaker.write_edward_model(file_prefix, bnet, vtx_to_data=vtx_to_data,
                                sample_size=batch_size)

.py file with model can be found here

<a href="../../examples_cbnets/WetGrass_edward.py">
../../examples_cbnets/WetGrass_edward.py</a>

In [5]:
# -i option allows it to access notebook's namespace
%run -i examples_cbnets/WetGrass_edward.py

In [6]:
# Generate training data
#[Cloudy, WetGrass] = [0, 1] all the time
data_size = 1000
data = np.tile([0, 1], [data_size, 1])
data

# [Cloudy, WetGrass] = [0, 1] most of time
# data_size = 10000
# bern = np.random.binomial
# data = np.stack( 
#     [[bern(1, .1), bern(1, .9)] for _ in range(data_size)])
# data

array([[0, 1],
       [0, 1],
       [0, 1],
       ..., 
       [0, 1],
       [0, 1],
       [0, 1]])

In [7]:
num_epochs = 10 # number of full passes, cycles, over data
num_batches = data_size//batch_size
assert batch_size*num_batches == data_size 

In [8]:
q_dict = {Rain: Rain_q, Sprinkler: Sprinkler_q}
data_dict = {Cloudy: Cloudy_ph, WetGrass: WetGrass_ph}
inf = edi.KLqp(q_dict, data=data_dict)
n_iter = num_batches*num_epochs
scale = {Cloudy:num_batches, WetGrass:num_batches}
inf.initialize(n_iter=n_iter, n_samples=5, scale=scale)

tf.global_variables_initializer().run()

bgen = mm.ModelMaker.batch_gen([data[:, 0], data[:,1]], batch_size)
for i in range(inf.n_iter):
    Cloudy_bat, WetGrass_bat = next(bgen)
    info_dict = inf.update({Cloudy_ph: Cloudy_bat, WetGrass_ph: WetGrass_bat})
    inf.print_progress(info_dict)
inf.finalize()

100/100 [100%] ██████████████████████████████ Elapsed: 4s | Loss: 703.859


In [9]:
# print final prob distributions

print('\nRain')
print(Rain_q.probs.eval())

print('\nSprinkler')
print(Sprinkler_q.probs.eval())


Rain
[ 0.03795076  0.96204931]

Sprinkler
[ 0.033005  0.966995]
